# CIS-PD Form Data

Use datadictionary and codelist to convert the scores on each form. Save each file in a pickle format.

In [23]:
# import packages
import os
import platform # don't need
import pandas as pd
import re
import h5py #save hdf5 but pandas has a function...
import numpy as np

## Clean up data dictionary and codelist

In [2]:
dd = pd.read_sas('Y:\CIS-PD MUSC\datadictionary.sas7bdat')
dd.columns = dd.columns.str.replace('z', '')
del dd['CRFNb']
del dd['FieldNm']
del dd['SASNm']
del dd['SASLabel']
del dd['FieldTypeNm']
del dd['Length']
del dd['Unit']
del dd['Required']
dd = dd.dropna()
dd = dd.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)

In [3]:
codelist = pd.read_sas('Y:\CIS-PD MUSC\codelist.sas7bdat')
codelist.columns = codelist.columns.str.replace('z', '')
codelist = codelist.rename(columns={'GroupID': 'CodeGroup'})
codelist = codelist.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)

In [4]:
# Merge data dictionary and codelist
result = dd.merge(codelist, left_on='CodeGroup', right_on='CodeGroup', how='left')
result.FieldNb = result.FieldNb.str.replace('z', '')

In [4]:
path = r'Y:\CIS-PD MUSC'
path2 = r'Y:\CIS-PD MUSC\decoded_forms'

## Change code to value function

In [6]:
def CodetoValue(FormNb, FormSeries, QNm):
    resultFilt = result[(result.FormID == FormNb) & (result.FieldNb == QNm)]
    return np.vstack([resultFilt[resultFilt['ItemNb']==INb]['ItemNm'].values if INb>=0.0 else INb for INb in FormSeries])

In [7]:
formlist = list(['form101','form126','form238','form239','form240','form241','form242','form243',
               'form501','form502','form503','form504','form505','form506','form508','form509',
               'form510','form515','form518'])

In [ ]:
for f in formlist:
    temporiginal = pd.read_sas(os.path.join(path,f + '.sas7bdat'))
    f2 = f.replace('orm','')
    f2 = f2.capitalize()
    temporiginal.columns = temporiginal.columns.str.replace('z', '')
    temporiginal.columns = temporiginal.columns.str.replace(f2, '')
    temp = temporiginal.copy()
    temp = temp.filter(regex='Q|DataCollected')
    temp = temp.select_dtypes(include=['float64'])
    colnames = list(temp)
    form = f
    form = form.replace('form', '')
    form = np.float64(form)
    
    for c in colnames:
        if sum((result.FormID == form) & (result.FieldNb == c))==0:  continue
        temp[c] = CodetoValue(form, temp[c],c)
    
    temporiginal.update(temp)
    temporiginal = temporiginal.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)
    
    # save file
    filename = os.path.join(path2,f + '.h5')
    temporiginal.to_hdf(filename,'data')